In [ ]:
import numpy as np
from numpy import random
from numpy import math
import matplotlib  
import matplotlib.pyplot as plt 

# copy
import copy


# import Keras and TF
import tensorflow as tf
from tensorflow import keras

from tensorflow.python.keras.models import clone_model
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Add, Dense, Activation, Flatten, Conv2D, Conv1D, MaxPooling2D, Dropout,BatchNormalization, Input, concatenate, Lambda
from tensorflow.python.keras.callbacks import Callback
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator


from numpy import linalg as LA
from tensorflow.python.keras import backend as K

In [ ]:
# function for checking if an equtorial triangle (H1, H2, H3) is acute or obtuse: accute = 0 and obtuse = +/- 2pi
def acute_obtuse_triangle(H1, H2, H3):

    # V1
    H_matrix = H1[0] * sigmax + H1[1] * sigmay + H1[2] * sigmaz
    eigenValues, eigenVectors = LA.eig(H_matrix)
    idx = eigenValues.argsort()[::-1]   
    U1eigenVectors = eigenVectors[:,idx] 

    # V2
    H_matrix = H2[0] * sigmax + H2[1] * sigmay + H2[2] * sigmaz
    eigenValues, eigenVectors = LA.eig(H_matrix)
    idx = eigenValues.argsort()[::-1]   
    U2eigenVectors = eigenVectors[:,idx]  


    # V3
    H_matrix = H3[0] * sigmax + H3[1] * sigmay + H3[2] * sigmaz
    eigenValues, eigenVectors = LA.eig(H_matrix)
    idx = eigenValues.argsort()[::-1]   
    U3eigenVectors = eigenVectors[:,idx]  

    U12 = np.matmul(U1eigenVectors[:,0].conj().T, U2eigenVectors[:,0])
    U23 = np.matmul(U2eigenVectors[:,0].conj().T, U3eigenVectors[:,0])          
    U31 = np.matmul(U3eigenVectors[:,0].conj().T, U1eigenVectors[:,0])
            

    return 2 * np.angle(U12 * U23 * U31)

In [ ]:
# function for calculating the Chern number
def Chern_number(state, N_k): 
    
    s = 1/(N_k)**2  
    
    CN = 0
   
    
    
    # sum over the triangular grid
    
    for i in range(N_k) : 
        for j in range(N_k) :
        
            # V1
            state_matrix = state[i, j, 0] * sigmax + state[i, j, 1] * sigmay + state[i, j, 2] * sigmaz
            eigenValues, eigenVectors = LA.eig(state_matrix)
            idx = eigenValues.argsort()[::-1]   
            U1eigenVectors = eigenVectors[:,idx]     
            
            # V2
            state_matrix = state[i, j - 1, 0] * sigmax + state[i, j - 1, 1] * sigmay + state[i, j - 1, 2] * sigmaz
            eigenValues, eigenVectors = LA.eig(state_matrix)
            idx = eigenValues.argsort()[::-1]   
            U2eigenVectors = eigenVectors[:,idx]   
            
            # V3
            state_matrix = state[i - 1, j - 1, 0] * sigmax + state[i - 1, j - 1, 1] * sigmay + state[i - 1, j - 1, 2] * sigmaz
            eigenValues, eigenVectors = LA.eig(state_matrix)
            idx = eigenValues.argsort()[::-1]   
            U3eigenVectors = eigenVectors[:,idx] 
            
            
            # V4
            state_matrix = state[i - 1, j, 0] * sigmax + state[i - 1, j, 1] * sigmay + state[i - 1, j, 2] * sigmaz
            eigenValues, eigenVectors = LA.eig(state_matrix)
            idx = eigenValues.argsort()[::-1]   
            U4eigenVectors = eigenVectors[:,idx] 
            
            
            
            
            # Calculate relative phases between the neighboring sites (with triangular grid as in Fig. )
            U12 = np.matmul(U1eigenVectors[:,0].conj().T, U2eigenVectors[:,0])
            U23 = np.matmul(U2eigenVectors[:,0].conj().T, U3eigenVectors[:,0])
            U34 = np.matmul(U3eigenVectors[:,0].conj().T, U4eigenVectors[:,0])
            U41 = np.matmul(U4eigenVectors[:,0].conj().T, U1eigenVectors[:,0])
            
            U31 = np.matmul(U3eigenVectors[:,0].conj().T, U1eigenVectors[:,0])
            U13 = np.matmul(U1eigenVectors[:,0].conj().T, U3eigenVectors[:,0])
            
            # Closed loop 1: triangle 1
            Flux =  np.angle(U12 * U23 * U31)
            CN += Flux 
        
            # Closed loop 2: triangle 2
            Flux =  np.angle(U13 * U34 * U41)
            CN += Flux
            
 
            
    return CN / (2 * math.pi)         

# Pauli matrices
sigmax = np.array([[0, 1], [1, 0]], dtype=complex)
sigmay = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigmaz = np.array([[1, 0], [0, -1]], dtype=complex)
sigma0 = np.array([[1, 0], [0, 1]], dtype=complex)

# extract Pauli components
def ToSigma0(H):
    
    f = np.real((H[0,0] + H[1,1])/2)
    
    return f

def ToSigmaZ(H):
    
    f = np.real((H[0,0] - H[1,1])/2)
    
    return f

def ToSigmaX(H):
    
    f = (np.real(H[0,1]) + np.real(H[1,0]))/2
    
    return f

def ToSigmaY(H):
    
    f = (np.imag(H[0,1]) - np.imag(H[1,0]))/2
    
    return f


# transform states from vector form to spherical coordinates and vice versa
def sphere_form(H):
    
    # norm
    E = (H[0]**2 + H[1]**2 + H[2]**2)**0.5
        
    # spherical angles
    theta_state = np.arccos(H[2]/E)
    phi_state = math.atan2(H[1], H[0])            
    
    return phi_state, theta_state



def from_sphere(phi_state, theta_state):

    H = np.zeros((3), dtype = float)
    
    #x, y, z
    H[0] = math.sin(theta_state) * math.cos(phi_state)
    H[1] = math.sin(theta_state) * math.sin(phi_state)
    H[2] = math.cos(theta_state)
    
    return H

        
# construct states: trvial, C = 0, C = 1 and C = random
def A_state_trivial(N_k):
        
    H = np.zeros((N_k, N_k, 3), dtype = float)
        
    for i_x in range(N_k) : 
        for i_y in range(N_k) : 
            
            k_x = 2 * math.pi * i_x/ N_k
            k_y = 2 * math.pi * i_y/ N_k
                
            H[i_x, i_y, 0] = 0
            H[i_x, i_y, 1] = 0
            H[i_x, i_y, 2] = 1
            
            E = (H[i_x, i_y, 0]**2 + H[i_x, i_y, 1]**2 + H[i_x, i_y, 2]**2)**0.5
            
            # normilize
            H[i_x, i_y, 0] /= E
            H[i_x, i_y, 1] /= E
            H[i_x, i_y, 2] /= E
            
    return H


def A_state_C_0(N_k):
        
    H = np.zeros((N_k, N_k, 3), dtype = float)
        
    for i_x in range(N_k) : 
        for i_y in range(N_k) : 
            
            k_x = 2 * math.pi * i_x/ N_k
            k_y = 2 * math.pi * i_y/ N_k
                
            H[i_x, i_y, 0] = math.sin(k_x)
            H[i_x, i_y, 1] = math.sin(k_y)
            H[i_x, i_y, 2] = 3 + math.cos(k_x) + math.cos(k_y)
            
            E = (H[i_x, i_y, 0]**2 + H[i_x, i_y, 1]**2 + H[i_x, i_y, 2]**2)**0.5
            
            # normilize
            H[i_x, i_y, 0] /= E
            H[i_x, i_y, 1] /= E
            H[i_x, i_y, 2] /= E
            
    return H


def A_state_C_1(N_k):
        
    H = np.zeros((N_k, N_k, 3), dtype = float)
        
    for i_x in range(N_k) : 
        for i_y in range(N_k) : 
            
            k_x = 2 * math.pi * i_x/ N_k
            k_y = 2 * math.pi * i_y/ N_k
                
            H[i_x, i_y, 0] = math.sin(k_x)
            H[i_x, i_y, 1] = math.sin(k_y)
            H[i_x, i_y, 2] = 1.5 + math.cos(k_x) + math.cos(k_y)
            
            E = (H[i_x, i_y, 0]**2 + H[i_x, i_y, 1]**2 + H[i_x, i_y, 2]**2)**0.5
            
            # normilize
            H[i_x, i_y, 0] /= E
            H[i_x, i_y, 1] /= E
            H[i_x, i_y, 2] /= E
            
    return H

def A_state_random(N_k):
        
    H = np.zeros((N_k, N_k, 3), dtype = float)
        
    for i_x in range(N_k) : 
        
        for i_y in range(N_k) : 
            
            H[i_x, i_y, 0] = random.random() - 0.5
            H[i_x, i_y, 1] = random.random() - 0.5
            H[i_x, i_y, 2] = random.random() - 0.5
            
            E = (H[i_x, i_y, 0]**2 + H[i_x, i_y, 1]**2 + H[i_x, i_y, 2]**2)**0.5
            
            # normilize
            H[i_x, i_y, 0] /= E
            H[i_x, i_y, 1] /= E
            H[i_x, i_y, 2] /= E
            
    return H


# continuity check for a grid triangle (following App. B)
def continuity_check(v1, v2, v3, delta_phi):
    
    # rotate vector v3 around z-axis by delta_phi and check if this condition is continuous
    n = np.cross(v1, v2)
    
    phi_state, theta_state = sphere_form(v3)
    
    A = math.sin(theta_state) * n[0]
    B = math.sin(theta_state) * n[1]
    C = math.cos(theta_state) * n[2]
    
    a = A**2 + B**2
    b = 2 * A * C
    c = C**2 - B**2
    
    D = b**2 - 4 * a * c
    
 
    # solve for cos(phi_x)
    if D < 0:
        return True # this solution means that the planes of interest do not intersect -> the deformation is continuous
    else:
        x1 = (- b - D**0.5)/(2 * a)
        x2 = (- b + D**0.5)/(2 * a)
       
    # consider these two solutions separately
    if abs(x1) <= 1:
        cosX = x1
        sinX = -(A * cosX + C)/B
        
        if B == 0:
            phiX = math.atan2(np.sign(C), 0) # angle from -pi to pi
        else:
            phiX = math.atan2(sinX, cosX)  # angle from -pi to pi
        
        # vector form of v_x
        vX1 = from_sphere(phiX, theta_state)
        
        # check if the triangle defined by v1, v2, and vX1 is acute (if the spherical triangle area is 2 pi)
        SA = acute_obtuse_triangle(v1, v2, vX1)  # 0 or +/- 2 pi
        
        epsilon = 0.01 # numeric error cutoff
        if abs(abs(SA) - 2*math.pi) < epsilon:
            
            # check if the considered deformation passes this "discontunuous" point
            
            phiX_relative = phiX - phi_state # angle of phiX counted from phi_state = 0
            
            if phiX_relative > math.pi:
                phiX_relative -= 2*math.pi
            if phiX_relative < (-math.pi):
                phiX_relative += 2*math.pi
                
            if (delta_phi > 0) and (phiX_relative > 0) and (delta_phi > phiX_relative):
                return False
            
            if (phiX_relative < 0) and (phiX_relative < 0) and (delta_phi < phiX_relative):
                return False
            
    if abs(x2) <= 1:
        cosX = x2
        sinX = -(A * cosX + C)/B
        
        if B == 0:
            phiX = math.atan2(np.sign(C), 0) # angle from -pi to pi
        else:
            phiX = math.atan2(sinX, cosX)  # angle from -pi to pi
        
        # vector form of v_x
        vX2 = from_sphere(phiX, theta_state)
        
        # check if the triangle defined by v1, v2, and vX1 is acute (if the spherical triangle area is 2 pi)
        SA = acute_obtuse_triangle(v1, v2, vX2)  # 0 or +/- 2 pi 
        
        epsilon = 0.01 # numeric error cutoff
        if abs(abs(SA) - 2*math.pi) < epsilon:
            
            # check if the considered deformation passes this "discontunuous" point
            
            phiX_relative = phiX - phi_state # angle of phiX counted from phi_state = 0
            
            if phiX_relative > math.pi:
                phiX_relative -= 2*math.pi
            if phiX_relative < (-math.pi):
                phiX_relative += 2*math.pi
                
            if (delta_phi > 0) and (phiX_relative > 0) and (delta_phi > phiX_relative):
                return False
            
            if (phiX_relative < 0) and (phiX_relative < 0) and (delta_phi < phiX_relative):
                return False
        
    return True

# rotation matrix over angle phi around axis x, y, or z  
def matrix_rot(phi, axis):

    c = math.cos(phi)
    s = math.sin(phi)


    R1 = np.array([[ c, -s, 0], [s,  c, 0], [0,   0,  1]])
    R2 = np.array([[ c, 0,  s], [0,  1,  0], [-s, 0,  c]])
    R3 = np.array([[ 1, 0,   0], [0, c, -s], [0, s,  c]])

    if axis == 0:
        return R1
    
    if axis == 1:
        return R2
    
    if axis == 2:
        return R3

    
# perform a 3d random rotation of the whole system and 'N_submoves' basic random deformations
def make_move(H, N_k, N_submoves):
    
  
    # rotate over random angle around x-axis (-pi, pi)
    phi1  = (2 * math.pi * random.random() - math.pi)
    matrix1 = matrix_rot(phi1, 0)
    
    # rotate over random angle around y-axis (-pi, pi)
    phi2  = (2 * math.pi * random.random() - math.pi)
    matrix2 = matrix_rot(phi2, 1)
    
    
    # rotate over random angle around z-axis (-pi, pi)
    phi3  = (2 * math.pi * random.random() - math.pi)
    matrix3 = matrix_rot(phi3, 2)
    
    
    # rotation of the whole system
    Hcopy = copy.deepcopy(H)
    for i_x in range(N_k): 
        for i_y in range(N_k):  
    
    
            H1 = np.matmul(matrix1, H[i_x, i_y, :])
            H1 = np.matmul(matrix2, H1)
            Hcopy[i_x, i_y, :]  = np.matmul(matrix3, H1)
    
    # rotate over a random angle around z-axis and check for continuity
    for step_n in range(N_submoves):
    
        # to start the loop 
        continuous_condition = False
        
        while continuous_condition == False:
        
            #--------- initialize the deformation-------------
        
            # Continuous = True     
            continuous_condition = True
        
        
            # rotate over a random angle around z-axis (-pi, pi)
            delta_phi  = (2 * random.random() - 1)* math.pi
      
            # choose the deformation center point 
            i_x = random.randint(0, N_k - 1)
            i_y = random.randint(0, N_k - 1)
                

            # neighboring grid points (periodic boundary condition)
            i_x_p = i_x + 1
            i_y_p = i_y + 1
        
            if i_x_p == N_k :
                i_x_p = 0
            if i_y_p == N_k :
                i_y_p = 0

                
            # check for continuity  (6 grid triangles, cf. article)
                
            # 1. 
            H1 = Hcopy[i_x-1, i_y-1, :]
            H2 = Hcopy[i_x, i_y-1, :]                    
            H3 = Hcopy[i_x, i_y, :]
                
            if continuity_check(H1, H2, H3, delta_phi) == False:
                continuous_condition = False
            
            # 2.
            H1 = Hcopy[i_x-1, i_y-1, :]
            H2 = Hcopy[i_x-1, i_y, :]                    
            H3 = Hcopy[i_x, i_y, :]
                
            if continuity_check(H1, H2, H3, delta_phi) == False:
                continuous_condition = False
                   
            # 3.
            H1 = Hcopy[i_x_p, i_y_p, :]
            H2 = Hcopy[i_x_p, i_y, :]                    
            H3 = Hcopy[i_x, i_y, :]
                
            if continuity_check(H1, H2, H3, delta_phi) == False:
                continuous_condition = False
            
            # 4.
            H1 = Hcopy[i_x_p, i_y_p, :]
            H2 = Hcopy[i_x, i_y_p, :]                    
            H3 = Hcopy[i_x, i_y, :]
                
            if continuity_check(H1, H2, H3, delta_phi) == False:
                continuous_condition = False
            
                
            # 5. 
            H1 = Hcopy[i_x - 1, i_y, :]
            H2 = Hcopy[i_x, i_y_p, :]                    
            H3 = Hcopy[i_x, i_y, :]
                
            if continuity_check(H1, H2, H3, delta_phi) == False:
                continuous_condition = False
            
            # 6. 
            H1 = Hcopy[i_x_p, i_y, :]
            H2 = Hcopy[i_x, i_y-1, :]                    
            H3 = Hcopy[i_x, i_y, :]
                
            if continuity_check(H1, H2, H3, delta_phi) == False:
                continuous_condition = False
                    
        
        #--------- Perform the deformation-------------
                    
        phi_state, theta_state  = sphere_form(Hcopy[i_x, i_y, :])              
        phi_state =  phi_state + delta_phi     
        Hcopy[i_x, i_y, :] = from_sphere(phi_state, theta_state)
        
        
    return Hcopy

# create the datasets for the training:
def create_data(state, N_samples, N_k, N_moves, N_submoves): 

    # Chern number (just to verify)
    print('-----------------------------------')
    print(Chern_number(state, N_k))
    
    # Generate random data
    generated_states = np.zeros((N_samples, N_k + 1, N_k + 1, 3), dtype = 'float')
    
    for step in range(N_samples):
    
        state_new = copy.deepcopy(state)
        
        for move_per_step in range(N_moves) :
                
            # make a move
            state_new = make_move(state_new, N_k, N_submoves)
            
        # Chern number (just to verify)
        print(Chern_number(state_new, N_k))
        
        # save
        generated_states[step, :N_k, :N_k, :] = state_new
        
        #impose periodic boundary conditions
        generated_states[step, N_k, 0, :] = generated_states[step, 0, 0, :] 
        generated_states[step, 0, N_k, :] = generated_states[step, 0, 0, :] 
        generated_states[step, N_k, N_k, :] = generated_states[step, 0, 0, :]
        
        generated_states[step, N_k, :, :] = generated_states[step, 0, :, :] 
        generated_states[step, :, N_k, :] = generated_states[step, :, 0, :] 
        
        #winding (just to verify)
        print(step)
        
    return generated_states
    

In [ ]:
# produce the datasets
N_moves = 50
N_submoves = 20
N_k = 10
N_sample = 10000

#--------------------------------------------
# trivial
state = A_state_trivial(N_k)
generated_states = create_data(state, N_sample, N_k, N_moves, N_submoves)

Path = 'data_A\\trivial.npy'
np.save(Path,  generated_states)

#--------------------------------------------
# case C = 0 
state = A_state_C_0(N_k)
generated_states = create_data(state, N_sample, N_k, N_moves, N_submoves)

Path = 'data_A\\C_0.npy'
np.save(Path,  generated_states)

#--------------------------------------------
# case C = 1
state = A_state_C_1(N_k)
generated_states = create_data(state, N_sample, N_k, N_moves, N_submoves)

Path = 'data_A\\C_1.npy'
np.save(Path,  generated_states)

#--------------------------------------------
# case C = random
state = A_state_random(N_k)
generated_states = create_data(state, N_sample, N_k, N_moves, N_submoves)

Path = 'data_A\\C_random.npy'
np.save(Path,  generated_states)



In [ ]:
# interpolate from (N_k + 1, N_k + 1)  to (2*N_k + 1, 2*N_k + 1) k-points
def expend(data):
    
    N_k = np.shape(data)[1]  - 1   
    
    data_new = np.zeros((data.shape[0], 2*N_k + 1, 2*N_k + 1, 3), dtype = float)
    
    for n in range(data.shape[0]):
        for i_x in range(N_k + 1):    
            for i_y in range(N_k + 1): 
                data_new[n, 2*i_x, 2*i_y, :] = data[n, i_x, i_y, :]
    
        for i_x in range(N_k):
            for i_y in range(N_k + 1): 
                data_new[n, 2*i_x + 1, 2*i_y, :] = interpolate(data_new[n, 2*i_x, 2*i_y, :], data_new[n, 2*i_x + 2, 2*i_y, :])
       
        for i_x in range(N_k + 1):
            for i_y in range(N_k): 
                data_new[n, 2*i_x, 2*i_y + 1, :] = interpolate(data_new[n, 2*i_x, 2*i_y, :], data_new[n, 2*i_x, 2*i_y + 2, :])       
            
      
        for i_x in range(N_k):
            for i_y in range(N_k): 
                data_new[n, 2*i_x + 1, 2*i_y + 1, :] = interpolate(data_new[n, 2*i_x, 2*i_y, :], data_new[n, 2*i_x + 2, 2*i_y + 2, :])        
            
    return data_new 


# interpolation between two vectors      
def interpolate(H1, H2):
    
    H_int = (H1 + H2)/np.linalg.norm(H1 + H2)
        
    return H_int
 

In [ ]:
# do the interpolation and save the data

Path = 'data_A\\trivial.npy'
data = np.load(Path)

expended_data = expend(data)

Path = 'data_A_expended\\trivial.npy'
np.save(Path,  expended_data)

#--------------------------------------------
Path = 'data_A\\C_0.npy'
data = np.load(Path)

expended_data = expend(data)

Path = 'data_A_expended\\C_0.npy'
np.save(Path,  expended_data)

#--------------------------------------------
Path = 'data_A\\C_1.npy'
data = np.load(Path)

expended_data = expend(data)

Path = 'data_A_expended\\C_1.npy'
np.save(Path,  expended_data)

#--------------------------------------------
Path = 'data_A\\C_random.npy'
data = np.load(Path)

expended_data = expend(data)

Path = 'data_A_expended\\C_random.npy'
np.save(Path,  expended_data)